In [ ]:
import torch
import torch.optim as optim
import data_generator
import argparse
from torch.autograd import Variable
import numpy as np
import random
torch.set_num_threads(2)
import os
import re
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [ ]:
parser = argparse.ArgumentParser(description = 'data generation')
parser.add_argument('--A_n', type = int, default = 7876,
               help = 'number of author node')
parser.add_argument('--P_n', type = int, default = 14518,
               help = 'number of paper node')
parser.add_argument('--V_n', type = int, default =38542 ,
               help = 'number of venue node')
#parser.add_argument('--C_n', type = int, default = 4,
#			   help = 'number of node class label')
parser.add_argument('--data_path', type = str, default = r'D:\11.12_Topic_Recommendation\data',
				   help='path to data')
parser.add_argument('--model_path', type = str, default = r'D:\11.12_Topic_Recommendation\model',
				   help='path to save model')
parser.add_argument('--walk_n', type = int, default = 10,
			   help='number of walk per root node')
parser.add_argument('--walk_L', type = int, default = 10,
			   help='length of each walk')
parser.add_argument('--window', type = int, default = 7,
			   help='window size for relation extration')
parser.add_argument('--T_split', type = int, default = 2023,
			   help = 'split time of train/test data')
parser.add_argument('--in_f_d', type = int, default = 128,
                       help = 'input feature dimension')
parser.add_argument('--embed_d', type = int, default = 128,
                       help = 'embedding dimension')
parser.add_argument('--lr', type = int, default = 0.001,
                       help = 'learning rate')
parser.add_argument('--batch_s', type = int, default = 20000,
               help = 'batch size')
parser.add_argument('--mini_batch_s', type = int, default = 200,
               help = 'mini batch size')
parser.add_argument('--train_iter_n', type = int, default = 50,
               help = 'max number of training iteration')
parser.add_argument("--random_seed", default = 10, type = int)
parser.add_argument('--train_test_label', type= int, default = 0,
               help='train/test label: 0 - train, 1 - test, 2 - code test/generate negative ids for evaluation')
parser.add_argument('--save_model_freq', type = float, default = 2,
               help = 'number of iterations to save model')
parser.add_argument("--cuda", default = 0, type = int)
parser.add_argument("--checkpoint", default = '', type=str)

args = parser.parse_known_args()[0]
print(args)
random.seed(args.random_seed)
np.random.seed(args.random_seed)
torch.manual_seed(args.random_seed)
torch.cuda.manual_seed_all(args.random_seed)

In [ ]:
#Load Edges

a_p_list_train = [[] for k in range(args.A_n)]
a_p_list_test = [[] for k in range(args.A_n)]
p_a_list_train = [[] for k in range(args.P_n)]
p_a_list_test = [[] for k in range(args.P_n)]
#p_p_cite_list_train = [[] for k in range(self.args.P_n)]
#p_p_cite_list_test = [[] for k in range(self.args.P_n)]
v_p_list_train = [[] for k in range(args.V_n)]

#relation_f = ["a_p_list_train.txt", "a_p_list_test.txt", "p_a_list_train.txt", "p_a_list_test.txt",\
# "p_p_cite_list_train.txt", "p_p_cite_list_test.txt", "v_p_list_train.txt"]
relation_f = [r"\a_p_list_train.txt", r"\a_p_list_test.txt", r"\p_a_list_train.txt", r"\p_a_list_test.txt",\
  r"\v_p_list_train.txt"]

#store academic relational data 
for i in range(len(relation_f)):
    f_name = relation_f[i]
    neigh_f = open(args.data_path + f_name, "r")
    for line in neigh_f:
        line = line.strip()
        node_id = int(re.split(':', line)[0])
        neigh_list = re.split(':', line)[1]
        neigh_list_id = re.split(',', neigh_list)
        if f_name == r'\a_p_list_train.txt':
            for j in range(len(neigh_list_id)):
                a_p_list_train[node_id].append('p'+str(neigh_list_id[j]))
        elif f_name == r'\a_p_list_test.txt':
            for j in range(len(neigh_list_id)):
                a_p_list_test[node_id].append('p'+str(neigh_list_id[j]))
        elif f_name == r'\p_a_list_train.txt':
            for j in range(len(neigh_list_id)):
                p_a_list_train[node_id].append('a'+str(neigh_list_id[j]))
        elif f_name == r'\p_a_list_test.txt':
            for j in range(len(neigh_list_id)):
                p_a_list_test[node_id].append('a'+str(neigh_list_id[j]))
        #elif f_name == 'p_p_cite_list_train.txt':
        #	for j in range(len(neigh_list_id)):
        #		p_p_cite_list_train[node_id].append('p'+str(neigh_list_id[j]))
        #elif f_name == 'p_p_cite_list_test.txt':
        #	for j in range(len(neigh_list_id)):
        #		p_p_cite_list_test[node_id].append('p'+str(neigh_list_id[j]))
        else:
            for j in range(len(neigh_list_id)):
                v_p_list_train[node_id].append('p'+str(neigh_list_id[j]))
    neigh_f.close()

In [ ]:
#Load Edges

p_v = [[] for k in range(args.P_n)]
p_v_f = open(args.data_path + r'\p_v.txt', "r")
for line in p_v_f:
    line = line.strip()
    node_id = int(re.split(':', line)[0])
    neigh_list = re.split(':', line)[1]
    neigh_list_id = re.split(',', neigh_list)
    for j in range(len(neigh_list_id)):
        p_v[node_id].append('v'+str(neigh_list_id[j]))
p_v_f.close()

In [ ]:
import six.moves.cPickle as pickle
import numpy as np
import string
import re
import random
import math
from collections import Counter
from itertools import *
import argparse
import pandas as pd

In [ ]:
#store paper content pre-trained embedding(abstract_title)
p_abstract_embed = np.zeros((args.P_n, args.in_f_d))
p_a_e_f = open(args.data_path + r"\p_abstract_embed.txt", "r")
for line in islice(p_a_e_f, 1, None):
    values = line.split()
    index = int(values[0])
    embeds = np.asarray(values[1:], dtype='float32')
    p_abstract_embed[index] = embeds
p_a_e_f.close()

p_title_embed = np.zeros((args.P_n, args.in_f_d))
p_t_e_f = open(args.data_path + r"\p_title_embed.txt", "r")
for line in islice(p_t_e_f, 1, None):
    values = line.split()
    index = int(values[0])
    embeds = np.asarray(values[1:], dtype='float32')
    p_title_embed[index] = embeds
p_t_e_f.close()

p_abstract_embed[1]

In [ ]:
#store structure pre-trained embedding
a_net_embed = np.zeros((args.A_n,args.in_f_d))
p_net_embed = np.zeros((args.P_n, args.in_f_d))
v_net_embed = np.zeros((args.V_n, args.in_f_d)) 
net_e_f = open(args.data_path + r"\node_net_embedding_10.txt", "r")
for line in islice(net_e_f, 1, None):
    line = line.strip()
    index = re.split(' ', line)[0]
    if len(index) and (index[0] == 'a' or index[0] == 'v' or index[0] == 'p'):
        embeds = np.asarray(re.split(' ', line)[1:], dtype='float32')
        #embeds = np.asarray([0]*128, dtype='float32')
        if index[0] == 'a':
            a_net_embed[int(index[1:])] = embeds
        elif index[0] == 'v':
            v_net_embed[int(index[1:])] = embeds
        else:
            p_net_embed[int(index[1:])] = embeds
net_e_f.close()

In [ ]:
#get structure embedding of P

p_a_net_embed = np.zeros((args.P_n, args.in_f_d))
for i in range(args.P_n):
    if len(p_a_list_train[i]):
        for j in range(len(p_a_list_train[i])):
            a_id = int(p_a_list_train[i][j][1:])
            p_a_net_embed[i] = np.add(p_a_net_embed[i], a_net_embed[a_id])
        p_a_net_embed[i] = p_a_net_embed[i] / len(p_a_list_train[i])

p_v_net_embed = np.zeros((args.P_n, args.in_f_d))
for i in range(args.P_n):
    if len(p_v[i]):
        for j in range(len(p_v[i])):
            v_id = int(p_v[i][j][1:])
            p_v_net_embed[i] = np.add(p_v_net_embed[i], v_net_embed[v_id])
        p_v_net_embed[i] = p_v_net_embed[i] / len(p_v[i])

In [ ]:
#empirically use 5 paper embedding for author content embeding generation
a_text_embed = np.zeros((args.A_n, args.in_f_d * 5))
for i in range(args.A_n):
    if len(a_p_list_train[i]):
        feature_temp = []
        if len(a_p_list_train[i]) >= 5:
            #id_list_temp = random.sample(a_p_list_train[i], 5)
            for j in range(5):
                feature_temp.append(p_abstract_embed[int(a_p_list_train[i][j][1:])])
        else:
            for j in range(len(a_p_list_train[i])):
                feature_temp.append(p_abstract_embed[int(a_p_list_train[i][j][1:])])
            for k in range(len(a_p_list_train[i]), 5):
                feature_temp.append(p_abstract_embed[int(a_p_list_train[i][-1][1:])])

        feature_temp = np.reshape(np.asarray(feature_temp), [1, -1])
        a_text_embed[i] = feature_temp

#empirically use 5 paper embedding for author content embeding generation
v_text_embed = np.zeros((args.V_n, args.in_f_d * 5))
for i in range(args.V_n):
    if len(v_p_list_train[i]):
        feature_temp = []
        if len(v_p_list_train[i]) >= 5:
            #id_list_temp = random.sample(a_p_list_train[i], 5)
            for j in range(5):
                feature_temp.append(p_abstract_embed[int(v_p_list_train[i][j][1:])])
        else:
            for j in range(len(v_p_list_train[i])):
                feature_temp.append(p_abstract_embed[int(v_p_list_train[i][j][1:])])
            for k in range(len(v_p_list_train[i]), 5):
                feature_temp.append(p_abstract_embed[int(v_p_list_train[i][-1][1:])])

        feature_temp = np.reshape(np.asarray(feature_temp), [1, -1])
        v_text_embed[i] = feature_temp

## GAT